In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df=pd.read_csv("/opt/ml/input/data/all_feature_data.csv")
df.head(4)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,cumAccuracy,tagCount,day,...,KnowledgeTagAcc,KTAccuracyCate,userLVbyTest,testLV,bigClassAcc,elapsedTime,tagCluster,elapsedTimeClass,seenCount,year
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.0,0,24,...,1.0,11,37.0,1.0,0.791908,3,3,1,0,2020
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,1.0,0,24,...,1.0,11,37.0,1.0,0.791908,8,3,2,0,2020
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,1.0,1,24,...,1.0,11,37.0,1.0,0.791908,7,3,2,0,2020
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,1.0,2,24,...,1.0,11,37.0,1.0,0.791908,7,3,2,0,2020


- assessmentItemID : drop
- testId : 최빈값
- Timestamp : drop
- KnowledgeTag : 최빈값
- answerCode -> 정답률
- day, month, year : drop
- bigClass : 최빈값
- userID : 그대로
- dataset : 그대로
- cumAccuracy : drop
- tagCount : 최빈 KT의 값
- cumCorrect : drop
- recAccuracy : 가장 마지막 값
- bigClassAccCate : 유저 한 명당 대분류별 정답률의 카테고리 -> 최빈 대분류의 정답률 카테고리
- KnowledgeTagAcc : 한 유저당 kt의정답률-> 최빈 KT의 정답률
- KTAccuracyCate : KT 정답률을 카테고리한 것 -> 최빈 KT의 카테고리
- userLVbyTest : 유저마다 한 개씩
- testLV : 최빈 시험지의 시험지 난이도
- bigClassAcc : 최빈 대분류의 정답률
- elapsedTime : 평균
- tagCluster : 최빈값
- elapsedTimeClass : 평균
- seenCount : drop 


In [3]:
userID=pd.unique(df.userID).tolist()
df=df.drop(columns=['assessmentItemID','Timestamp','day','month','year','cumAccuracy', 'seenCount','cumCorrect'])
df.head(3)

,userID,testId,answerCode,KnowledgeTag,dataset,tagCount,bigClass,recAccuracy,bigClassAccCate,KnowledgeTagAcc,KTAccuracyCate,userLVbyTest,testLV,bigClassAcc,elapsedTime,tagCluster,elapsedTimeClass
0,0,A060000001,1,7224,1,0,6,1.0,9,1.0,11,37.0,1.0,0.791908,3,3,1
1,0,A060000001,1,7225,1,0,6,1.0,9,1.0,11,37.0,1.0,0.791908,8,3,2
2,0,A060000001,1,7225,1,1,6,1.0,9,1.0,11,37.0,1.0,0.791908,7,3,2


In [4]:
new_df=list()
for user in tqdm(userID):
    data=list()
    temp=df[df['userID']==user]
    data.append(user) # 0 userId
    data.append(temp['testId'].mode().tolist()[0]) # 1 최빈 testId
    data.append(sum(temp['answerCode'])/len(temp['answerCode'])) # 2 전체 정답률
    data.append(int(temp['KnowledgeTag'].mode().tolist()[0])) # 3 최빈 knowledge tag
    data.append(int(temp[temp['KnowledgeTag']==data[3]].tagCount.iloc[-1])) # 4 최빈 knowledge tag가 나온 횟수
    data.append(temp[temp['KnowledgeTag']==data[3]].KnowledgeTagAcc.iloc[0]) # 5 최빈 KT의 정답률 
    data.append(temp[temp['KnowledgeTag']==data[3]].KTAccuracyCate.iloc[-1]) # 6 최빈 KT의 가장 마지막에 계산된 Acc카테고리
    data.append(temp[temp['KnowledgeTag']==data[3]].tagCluster.iloc[0]) # 7 최빈 KT의 카테고리
    data.append(temp['recAccuracy'].iloc[-1]) # 8 가장 최근의 정답률
    data.append(int(temp['bigClass'].mode().tolist()[0])) # 9 가장 많이 나온 대분류 값
    data.append(temp[temp['bigClass']==data[9]].bigClassAccCate.iloc[-1]) # 10 최빈 대분류의 가장 마지막에 계산된 Acc카테고리
    data.append(temp[temp['bigClass']==data[9]].bigClassAcc.iloc[-1]) # 11 최빈 대분류의 정답률
    data.append(int(temp[temp['testId']==data[1]].userLVbyTest.iloc[0])) # 12 최빈 testId의 유저 레벨
    data.append(int(temp[temp['testId']==data[1]].testLV.iloc[0])) # 13 최빈 testId의 레벨
    data.append(temp['elapsedTime'].mean()) # 14 걸린 시간의 평균
    data.append(temp['elapsedTimeClass'].mean()) # 15 걸린 시간의 카테고리 평균
    data.append(temp['dataset'].iloc[0]) 
    new_df.append(data)    
    
new_df=pd.DataFrame(data=new_df,columns=['userID', 'modeTestId','accuracy','modeKT','modeKTCount','modeKTAcc','modeKTAccCate','modeKTCate','recAuccracyLast','modeBigClass','modeBigClassAccCate',
                                        'modeBigClassAcc','userLVbyModeTest','modeTestLV','meanElapsedTime','meanElapsedTimeClass','dataset'])

new_df.head()

  0%|          | 2/7442 [00:00<02:34, 48.24it/s]


TypeError: cannot convert the series to <class 'int'>

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
new_df['modeTestId']=le.fit_transform(new_df['modeTestId'])
new_df.head()

In [ ]:
new_df.to_csv('all_feature_tabular_data.csv')